Meidan voter theorem says that if voter and candicates are distributed along a spectrum, any condorcet consistent voting method will elect the candidate preferred by the median voter. 

The simpliest situation is Voter rank candidate by proximity, and the number of voters is equally distributed along the spectrum. 

Here I want to mimic the decision making procedure and conclude how the median voter therem is concluded.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# 1) Setup: voters + candidates on a 1D spectrum
# -----------------------------
np.random.seed(0)

N_voters = 501
N_candidates = 9

# Simplest "equally distributed" voters: uniform grid on [0, 1]
voters = np.linspace(0, 1, N_voters)

# Candidates: put them on a grid too (you can randomize if you like)
candidates = np.linspace(0.05, 0.95, N_candidates)

# Median voter ideal point (for odd N, it's exact)
median_voter = np.median(voters)

# Candidate "closest to the median voter" (the theorem's prediction in discrete candidate sets)
pred_idx = np.argmin(np.abs(candidates - median_voter))


# -----------------------------
# 2) Preferences by proximity
#    Utility = -distance, so higher is better
# -----------------------------
# distances shape: (N_voters, N_candidates)
dist = np.abs(voters[:, None] - candidates[None, :])

# For each voter, rank candidates by increasing distance
# ranks[v, :] gives candidate indices sorted best->worst for voter v
ranks = np.argsort(dist, axis=1)


# -----------------------------
# 3) Pairwise majority / Condorcet matrix
#    wins[i, j] = number of voters who prefer i over j
# -----------------------------
wins = np.zeros((N_candidates, N_candidates), dtype=int)

# We can compute preference between i and j by comparing distances:
for i in range(N_candidates):
    for j in range(N_candidates):
        if i == j:
            continue
        wins[i, j] = np.sum(dist[:, i] < dist[:, j])  # strict preference
        # ties (equal distance) are ignored; with odd N and grids, ties are rare

# Majority threshold
maj = N_voters // 2 + 1

# i beats j if wins[i, j] >= maj
beats = wins >= maj

# Condorcet winner = candidate that beats everyone else
condorcet = np.where(np.all(beats | np.eye(N_candidates, dtype=bool), axis=1))[0]
condorcet_idx = condorcet[0] if len(condorcet) == 1 else None


print("Median voter ideal point:", median_voter)
print("Candidate closest to median voter (prediction): index", pred_idx, "position", candidates[pred_idx])

if condorcet_idx is None:
    print("No unique Condorcet winner found in this run.")
else:
    print("Condorcet winner: index", condorcet_idx, "position", candidates[condorcet_idx])


# -----------------------------
# 4) Visualization
# -----------------------------
# (A) Spectrum plot: voters + candidates + median voter + winners
plt.figure()
plt.scatter(voters, np.zeros_like(voters), s=8, alpha=0.3, label="Voters (ideal points)")
plt.scatter(candidates, np.ones_like(candidates)*0.1, s=80, marker="s", label="Candidates")

plt.axvline(median_voter, linestyle="--", label="Median voter")

plt.scatter(candidates[pred_idx], 0.1, s=200, marker="s", label="Closest-to-median candidate")
if condorcet_idx is not None:
    plt.scatter(candidates[condorcet_idx], 0.1, s=300, marker="*", label="Condorcet winner")

plt.yticks([])
plt.ylim(-0.05, 0.25)
plt.xlabel("Ideological spectrum")
plt.title("Median voter theorem (distance-based, single-peaked preferences)")
plt.legend()
plt.tight_layout()


# (B) Pairwise majority heatmap: who beats whom
plt.figure()
plt.imshow(wins, aspect="auto")
plt.colorbar(label="Voters preferring row over column")
plt.xticks(range(N_candidates), [f"{c:.2f}" for c in candidates], rotation=45)
plt.yticks(range(N_candidates), [f"{c:.2f}" for c in candidates])
plt.xlabel("Opponent candidate position")
plt.ylabel("Row candidate position")
plt.title("Pairwise majority counts (wins matrix)")
plt.tight_layout()


# (C) For each candidate, how many pairwise wins?
pairwise_wins_count = np.sum(beats, axis=1) - 1  # subtract self
plt.figure()
plt.bar(range(N_candidates), pairwise_wins_count)
plt.axhline(N_candidates - 1, linestyle="--", label="Beats everyone")
plt.xticks(range(N_candidates), [f"{c:.2f}" for c in candidates], rotation=45)
plt.ylabel("Number of pairwise wins")
plt.xlabel("Candidate position")
plt.title("Pairwise wins per candidate (Condorcet winner hits N-1)")
plt.legend()
plt.tight_layout()

plt.show()
